In [1]:
# Import libraries
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from scipy import spatial
from numpy.linalg import norm
from sklearn.neighbors import NearestNeighbors

In [2]:
# Load ViSim
data = pd.read_csv("datasets/ViSim-400/Visim-400.txt", sep = "\t")

In [3]:
# Load word2vec data
word2vec = {}
coordinates = []
with open("word2vec/W2V_150.txt", 'r', encoding='utf-8') as f:
    N = int(f.readline())
    dim = int(f.readline())
    for i in range(N):
        line = f.readline()
        elements = line.split()
        word2vec[elements[0]] = np.asanyarray([float(i) for i in elements[1:]], dtype=np.float32)
        coordinates.append(word2vec[elements[0]])
coordinates = np.array(coordinates)
list_word = np.array(list(word2vec.keys()))

In [38]:
# Init parameters
# Number of nearest neighbors
K = 10
# Word queried
w = "nhà"

In [28]:
# Mannual cosine metric
def cosine_dist(x, y):
    return 1 - np.dot(x, y) / norm(x) / norm(y)

In [30]:
# Model with metric of library
model = NearestNeighbors(n_neighbors=K, algorithm='brute', metric='cosine', n_jobs=-1)
model.fit(coordinates)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [31]:
# Model with mannual metric
nbrs = NearestNeighbors(n_neighbors=K, algorithm='brute', metric=cosine_dist, n_jobs=-1)
nbrs.fit(coordinates)

NearestNeighbors(algorithm='brute',
                 metric=<function cosine_dist at 0x000001789D9EF820>, n_jobs=-1,
                 n_neighbors=10)

In [33]:
# Find KNN with library metric model
model.kneighbors([word2vec[w]], K, return_distance=True)

(array([[5.9604645e-08, 2.8176874e-01, 3.7507415e-01, 3.8698512e-01,
         4.0976185e-01, 4.4179761e-01, 4.5175934e-01, 4.5987982e-01,
         4.6927774e-01, 4.6952456e-01]], dtype=float32),
 array([[   99, 29127, 12678, 48484, 74193, 69571, 10901, 10371,  6556,
         47989]], dtype=int64))

In [34]:
# Find KNN with mannual metric model
nbrs.kneighbors([word2vec[w]], K, return_distance=True)

(array([[0.        , 0.28176874, 0.37507415, 0.38698506, 0.40976185,
         0.4417976 , 0.45175928, 0.45987982, 0.46927774, 0.4695245 ]],
       dtype=float32),
 array([[   99, 29127, 12678, 48484, 74193, 69571, 10901, 10371,  6556,
         47989]], dtype=int64))

In [36]:
# %%timeit
res = np.dot(word2vec[w], coordinates.T) / norm(coordinates[np.where(list_word == w)]) / norm(coordinates, axis = 1)
res = 1 - res
index = np.argpartition(res, K)[:K]
print(res[index])
print(index)

[3.7507421e-01 4.0976179e-01 5.9604645e-08 2.8176874e-01 3.8698506e-01
 4.4179761e-01 4.6927768e-01 4.6952450e-01 4.5987982e-01 4.5175934e-01]
[12678 74193    99 29127 48484 69571  6556 47989 10371 10901]


In [37]:
# Print K nearest neighbor words
print(list_word[index])

['nhà_chung' 'nhà_ở' 'nhà' 'nhà_tầng' 'nhà_di_động' 'nhà_nguyên' 'khu_nhà'
 'nhà_trệt' 'nhà_trọ' 'nhà_cao_tầng']


In [41]:
# Euclidean distance
model = NearestNeighbors(n_neighbors=K, algorithm='brute', metric='euclidean', n_jobs=-1)
model.fit(coordinates)
dist, index = model.kneighbors([word2vec[w]], K, return_distance=True)
print(list_word[index])

[['nhà' 'nhà_nguyên' 'nhà_trệt' 'nhà_di_động' 'trên_nhà' '18,000' 'nàh'
  'ttxvn/vietnam' 'tay_không_bắt_giặc' 'nhà_phụ']]


In [44]:
# Dice distance
model = NearestNeighbors(n_neighbors=K, algorithm='brute', metric='dice', n_jobs=-1)
model.fit(coordinates)
dist, index = model.kneighbors([word2vec[w]], K, return_distance=True)
print(list_word[index])

[['băng_sơn' 'vũ_kỳ' 'tàng_thư' '2013-2014' 'lên_nước' 'cao_bá_quát'
  'ấm_dần_lên' 'tảng_băng_trôi' 'nonthaburi' 'khuyển']]


E:\UET\Lab\App\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:1765: DataConversionWarning: Data was converted to boolean for metric dice
  warnings.warn(msg, DataConversionWarning)


In [45]:
# Jaccard distance
model = NearestNeighbors(n_neighbors=K, algorithm='brute', metric='jaccard', n_jobs=-1)
model.fit(coordinates)
dist, index = model.kneighbors([word2vec[w]], K, return_distance=True)
print(list_word[index])

[['băng_sơn' 'vũ_kỳ' 'tàng_thư' '2013-2014' 'lên_nước' 'cao_bá_quát'
  'ấm_dần_lên' 'tảng_băng_trôi' 'nonthaburi' 'khuyển']]


E:\UET\Lab\App\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:1765: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
